### Postprocessing `.out` files

In [1]:
import re

In [2]:
def get_lines(filename):
    with open(filename) as f:
        lines = f.readlines()
    return lines

In [3]:
def extract_nums(string):
    remove_zeros = lambda x: x != 0
    lnums = [float(s) for s in re.findall(r'-?\d+\.?\d*', string)]
    return list(filter(remove_zeros, lnums))

In [4]:
def get_accuracy(filename):
    lines = get_lines(filename)
    for i, l in enumerate(lines):
        if 'Accuracy' in l:
            string_acc = ''.join(lines[i:])
            break
    return extract_nums(string_acc)

### Reader for `.out` files

In [5]:
import os
import pandas as pd
import itertools

In [6]:
def read(directory):
    root = '/'.join([os.getcwd(), directory])
    files = sorted(os.listdir(root))
    accs = {f[:-4]: get_accuracy('/'.join([root, f])) for f in files}
    return accs

In [7]:
def merge_ann_snn(accs):
    new_accs = {}
    for i, e in enumerate(accs):
        if i % 2 == 0:
            new_accs[e[:-4]] = accs[e]
        else:
            new_accs[e[:-4]].extend(accs[e])
    return new_accs

In [12]:
accs = merge_ann_snn(read(directory='20220523/eval'))

In [13]:
len(accs['resnet18_cifar100_LR01_L02'])

129

In [14]:
cols = ['ann']
for i in range(128):
    cols.append('t=' + str(i + 1))

In [16]:
cols[-1]

't=128'

In [17]:
df = pd.DataFrame.from_dict(accs, orient='index', columns=cols)
df_short = df[['ann', 't=1', 't=2', 't=4', 't=8', 't=16', 't=32', 't=64', 't=128']]

In [18]:
df_short

,ann,t=1,t=2,t=4,t=8,t=16,t=32,t=64,t=128
resnet18_cifar100_LR01_L02,0.7026,0.5756,0.6281,0.6627,0.6901,0.7019,0.7041,0.7048,0.7031
resnet18_cifar100_LR01_L04,0.7331,0.5088,0.5878,0.6569,0.7045,0.7268,0.7342,0.7351,0.7351
resnet18_cifar100_LR01_L08,0.7522,0.4036,0.5151,0.6173,0.6982,0.7373,0.7496,0.7522,0.7547
resnet18_cifar100_LR01_L16,0.7532,0.1132,0.3290,0.5246,0.6618,0.7249,0.7454,0.7516,0.7538
resnet18_cifar100_LR05_L02,0.7509,0.6494,0.6913,0.7217,0.7427,0.7493,0.7527,0.7512,0.7517
...,...,...,...,...,...,...,...,...,...
vgg16_cifar10_LR10_L16,0.9404,0.1114,0.7455,0.8738,0.9217,0.9357,0.9386,0.9398,0.9400
vgg16_cifar10_LR20_L02,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000
vgg16_cifar10_LR20_L04,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000
vgg16_cifar10_LR20_L08,0.9134,0.5286,0.6108,0.7842,0.8768,0.9044,0.9119,0.9131,0.9136


In [19]:
df.to_csv('logs_results_128_long.csv')
df_short.to_csv('logs_results_128_short.csv')